[List of French departments by life expectancy](https://en.wikipedia.org/wiki/List_of_French_departments_by_life_expectancy) / 
[Продолжительность жизни в департаментах Франции](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_департаментах_Франции)<br />
Data sources: [INSEE](https://www.insee.fr/fr/statistiques/2012749), [archive for 2019](https://web.archive.org/web/20210412152016/https://www.insee.fr/fr/statistiques/2012749)<br>
Data for departments are available at INSEE since 2015.

In [2]:
import pandas as pd
import math
import re

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [3]:
pd.options.display.min_rows = 6

In [4]:
def load_data_from_file_comma_decimal(file_name, year):
    df = pd.read_excel(f'data/{file_name}', sheet_name='DEP', skiprows=4, skipfooter=1, usecols="A:D", index_col='Unnamed: 0', na_values='nd',
                       decimal=',')
    df.rename(columns={'Unnamed: 1': 'department', 'Hom.':f'{year}_male', 'Fem.':f'{year}_female'}, inplace=True)
    df[f'{year}_fΔm'] = df[f'{year}_female'] - df[f'{year}_male']
    df[f'{year}_ar_mean'] = ((df[f'{year}_male'] + df[f'{year}_female']) / 2).round(2)  # ! without rounding it is possible error during sorting
    return df


df_2015 = load_data_from_file_comma_decimal('France-2015 (from 2017-08-27).xls', 2015)

In [5]:
def load_data_from_file(file_name, year):
    df = pd.read_excel(f'data/{file_name}', sheet_name='DEP', skiprows=4, skipfooter=1, usecols="A:D", index_col='Unnamed: 0', na_values='nd')
    df.rename(columns={'Unnamed: 1': 'department', 'Hom.':f'{year}_male', 'Fem.':f'{year}_female'}, inplace=True)
    df[f'{year}_fΔm'] = df[f'{year}_female'] - df[f'{year}_male']
    df[f'{year}_ar_mean'] = ((df[f'{year}_male'] + df[f'{year}_female']) / 2).round(2)  # ! without rounding it is possible error during sorting
    return df


df_2019 = load_data_from_file('France-2019.xls', 2019)
df_2024 = load_data_from_file('France-2024.xlsx', 2024)

In [6]:
df = pd.concat([df_2015, df_2019, df_2024.iloc[:, 1:]], axis='columns')

df.insert(loc=5, column='2015→2019', value=(df['2019_ar_mean']-df['2015_ar_mean']).round(2))
df.insert(loc=10, column='2019→2024', value=(df['2024_ar_mean']-df['2019_ar_mean']).round(2))
df['2015→2024'] = (df['2024_ar_mean']-df['2015_ar_mean']).round(2)

df.dropna(how='all', inplace=True)
df.fillna('')

,department,2015_male,2015_female,2015_fΔm,2015_ar_mean,2015→2019,department,2019_male,2019_female,2019_fΔm,2019→2024,2019_ar_mean,2024_male,2024_female,2024_fΔm,2024_ar_mean,2015→2024
01,Ain,80.4,85.5,5.1,82.95,0.5,Ain,80.9,86.0,5.1,0.25,83.45,81.4,86.0,4.6,83.70,0.75
02,Aisne,76.5,83.2,6.7,79.85,0.3,Aisne,76.9,83.4,6.5,0.55,80.15,77.7,83.7,6.0,80.70,0.85
03,Allier,78.1,84.5,6.4,81.30,-0.15,Allier,77.7,84.6,6.9,1.0,81.15,78.8,85.5,6.7,82.15,0.85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,La Réunion,77.1,83.6,6.5,80.35,1.25,La Réunion,78.5,84.7,6.2,-0.3,81.6,78.5,84.1,5.6,81.30,0.95
976,Mayotte,75.3,77.2,1.9,76.25,,Mayotte,,,,,,72.8,75.8,3.0,74.30,-1.95
F,France,79.0,85.1,6.1,82.05,0.6,France,79.7,85.6,5.9,0.15,82.65,80.0,85.6,5.6,82.80,0.75


In [7]:
# drop redundant records and sort DataFrame
df = df.drop(index=['P', '971', '972', '973', '974', '976', 'F'])  \
       .sort_values(by=['2024_ar_mean', '2024_male', '2024_female', '2019_ar_mean', ], ascending=False)

# place record about country as a whole to top of DataFrame
indexes = df.index.to_list()
ls_change_order = ['M']
indexes = list(el for el in indexes if el not in ls_change_order)
indexes = ls_change_order + indexes
df = df.reindex(indexes)

df

,department,2015_male,2015_female,2015_fΔm,2015_ar_mean,2015→2019,department,2019_male,2019_female,2019_fΔm,2019→2024,2019_ar_mean,2024_male,2024_female,2024_fΔm,2024_ar_mean,2015→2024
M,France métropolitaine,79.0,85.1,6.1,82.05,0.70,France métropolitaine,79.8,85.7,5.9,0.15,82.75,80.1,85.7,5.6,82.90,0.85
92,Hauts-de-Seine,81.5,86.2,4.7,83.85,0.70,Hauts-de-Seine,82.6,86.5,3.9,0.35,84.55,82.7,87.1,4.4,84.90,1.05
69,Rhône,80.3,85.9,5.6,83.10,0.85,Rhône,81.3,86.6,5.3,0.70,83.95,82.2,87.1,4.9,84.65,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Nièvre,77.1,84.1,7.0,80.60,-0.10,Nièvre,77.1,83.9,6.8,0.25,80.50,77.1,84.4,7.3,80.75,0.15
02,Aisne,76.5,83.2,6.7,79.85,0.30,Aisne,76.9,83.4,6.5,0.55,80.15,77.7,83.7,6.0,80.70,0.85
62,Pas-de-Calais,75.8,83.3,7.5,79.55,0.65,Pas-de-Calais,76.7,83.7,7.0,0.10,80.20,77.2,83.4,6.2,80.30,0.75


<br />
<br />

In [9]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df[['2015_ar_mean', '2015→2019', '2019_ar_mean', '2019→2024', '2024_ar_mean', '2015→2024']],
                       row_center=["13", "M"], nmb=3, max_lng=9)

Number of records: 97


,2015_ar_mean,2015→2019,2019_ar_mean,2019→2024,2024_ar_mean,2015→2024
max,83.85 -92,1.15 -2A,84.55 -92,1.05 -84,84.9 -92,1.75 -2A
max_2,83.8 -78,1.05 -49,84.45 -75,1.0 -03,84.65 -69,1.7 -84
max_3,83.8 -75,0.95 -77,84.25 -78,0.85 -81,84.55 -94,1.55 -69
13,– 82.25 –,– 0.6 –,– 82.85 –,– 0.5 –,– 83.35 –,– 1.1 –
M,– 82.05 –,– 0.7 –,– 82.75 –,– 0.15 –,– 82.9 –,– 0.85 –
min_3,80.0 -59,-0.25 -10,80.5 -80,-0.6 -11,80.75 -58,-0.55 -09
min_2,79.85 -02,-0.25 -09,80.2 -62,-0.65 -39,80.7 -02,-0.6 -79
min,79.55 -62,-0.45 -18,80.15 -02,-0.8 -79,80.3 -62,-0.7 -86


<br>
<br>

In [11]:
dd_replacement = {
    "F"   : {"fr": ("France", ""), "en": ("France on average", ""), "ru": ("Франция в целом", "")},
    "M"   : {"fr": ("France métropolitaine", ""), "en": ("Metropolitan France", ""), "ru": ("Метрополия Франции", "")},
    "01"   : {"fr": ("Ain", "Ain (département)"), "en": ("Ain", "Ain"), "ru": ("Эн", "Эн (департамент)")},
    "02"   : {"fr": ("Aisne", "Aisne (département)"), "en": ("Aisne", "Aisne"), "ru": ("Э́на", "Эна (департамент)")},
    "03"   : {"fr": ("Allier", "Allier (département)"), "en": ("Allier", "Allier"), "ru": ("Алье́", "Алье")},
    "04"   : {"fr": ("Alpes-de-Haute-Provence", "Alpes-de-Haute-Provence"), "en": ("Alpes-de-Haute-Provence", "Alpes-de-Haute-Provence"), "ru": ("Альпы Верхнего Прова́нса", "Альпы Верхнего Прованса")},
    "05"   : {"fr": ("Hautes-Alpes", "Hautes-Alpes"), "en": ("Hautes-Alpes", "Hautes-Alpes"), "ru": ("Верхние Альпы", "Верхние Альпы")},
    "06"   : {"fr": ("Alpes-Maritimes", "Alpes-Maritimes"), "en": ("Alpes-Maritimes", "Alpes-Maritimes"), "ru": ("Приморские Альпы", "Приморские Альпы (департамент)")},
    "07"   : {"fr": ("Ardèche", "Ardèche (département)"), "en": ("Ardèche", "Ardèche"), "ru": ("Арде́ш", "Ардеш (департамент)")},
    "08"   : {"fr": ("Ardennes", "Ardennes (département)"), "en": ("Ardennes", "Ardennes (department)"), "ru": ("Арде́нны", "Арденны (департамент)")},
    "09"   : {"fr": ("Ariège", "riège (département)"), "en": ("Ariège", "Ariège (department)"), "ru": ("Арье́ж", "Арьеж")},
    "10"   : {"fr": ("Aube", "Aube (département)"), "en": ("Aube", "Aube"), "ru": ("Об", "Об (департамент)")},
    "11"   : {"fr": ("Aude", "Aude (département)"), "en": ("Aude", "Aude"), "ru": ("Од", "Од (департамент)")},
    "12"   : {"fr": ("Aveyron", "Aveyron (département)"), "en": ("Aveyron", "Aveyron"), "ru": ("Аверо́н", "Аверон")},
    "13"   : {"fr": ("Bouches-du-Rhône", "Bouches-du-Rhône"), "en": ("Bouches-du-Rhône", "Bouches-du-Rhône"), "ru": ("Буш-дю-Ро́н", "Буш-дю-Рон")},
    "14"   : {"fr": ("Calvados", "Calvados (département)"), "en": ("Calvados", "Calvados (department)"), "ru": ("Кальвадо́с", "Кальвадос (департамент)")},
    "15"   : {"fr": ("Cantal", "Cantal (département)"), "en": ("Cantal", "Cantal"), "ru": ("Канта́ль", "Канталь")},
    "16"   : {"fr": ("Charente", "Charente (département)"), "en": ("Charente", "Charente"), "ru": ("Шара́нта", "Шаранта")},
    "17"   : {"fr": ("Charente-Maritime", "Charente-Maritime"), "en": ("Charente-Maritime", "Charente-Maritime"), "ru": ("Приморская Шара́нта", "Приморская Шаранта")},
    "18"   : {"fr": ("Cher", "Cher (département)"), "en": ("Cher", "Cher (department)"), "ru": ("Шер", "Шер (департамент)")},
    "19"   : {"fr": ("Corrèze", "Corrèze (département)"), "en": ("Corrèze", "Corrèze"), "ru": ("Корре́з", "Коррез")},
    "21"   : {"fr": ("Côte-d'Or", "Côte-d'Or"), "en": ("Côte-d'Or", "Côte-d'Or"), "ru": ("Кот-д’Ор", "Кот-д’Ор")},
    "22"   : {"fr": ("Côtes-d'Armor", "Côtes-d'Armor"), "en": ("Côtes-d'Armor", "Côtes-d'Armor"), "ru": ("Кот-д’Армор", "Кот-д’Армор")},
    "23"   : {"fr": ("Creuse", "Creuse (département)"), "en": ("Creuse", "Creuse"), "ru": ("Крёз", "Крёз (департамент)")},
    "24"   : {"fr": ("Dordogne", "Dordogne (département)"), "en": ("Dordogne", "Dordogne"), "ru": ("Дордо́нь", "Дордонь")},
    "25"   : {"fr": ("Doubs", "Doubs (département)"), "en": ("Doubs", "Doubs"), "ru": ("Ду", "Ду (департамент)")},
    "26"   : {"fr": ("Drôme", "Drôme (département)"), "en": ("Drôme", "Drôme"), "ru": ("Дром", "Дром (департамент)")},
    "27"   : {"fr": ("Eure", "Eure (département)"), "en": ("Eure", "Eure"), "ru": ("Эр", "Эр (департамент)")},
    "28"   : {"fr": ("Eure-et-Loir", "Eure-et-Loir"), "en": ("Eure-et-Loir", "Eure-et-Loir"), "ru": ("Эр и Луар", "Эр и Луар")},
    "29"   : {"fr": ("Finistère", "Finistère"), "en": ("Finistère", "Finistère"), "ru": ("Финистер", "Финистер")},
    "2A"   : {"fr": ("Corse-du-Sud", "Corse-du-Sud"), "en": ("Corse-du-Sud", "Corse-du-Sud"), "ru": ("Южная Ко́рсика", "Южная Корсика")},
    "2B"   : {"fr": ("Haute-Corse", "Haute-Corse"), "en": ("Haute-Corse", "Haute-Corse"), "ru": ("Верхняя Ко́рсика", "Верхняя Корсика")},
    "30"   : {"fr": ("Gard", "Gard"), "en": ("Gard", "Gard"), "ru": ("Гар", "Гар (департамент)")},
    "31"   : {"fr": ("Haute-Garonne", "Haute-Garonne"), "en": ("Haute-Garonne", "Haute-Garonne"), "ru": ("Верхняя Гаро́нна", "Верхняя Гаронна")},
    "32"   : {"fr": ("Gers", "Gers (département)"), "en": ("Gers", "Gers"), "ru": ("Жер", "Жер (департамент)")},
    "33"   : {"fr": ("Gironde", "Gironde (département)"), "en": ("Gironde", "Gironde"), "ru": ("Жиро́нда", "Жиронда")},
    "34"   : {"fr": ("Hérault", "Hérault (département)"), "en": ("Hérault", "Hérault"), "ru": ("Эро́", "Эро (департамент)")},
    "35"   : {"fr": ("Ille-et-Vilaine", "Ille-et-Vilaine"), "en": ("Ille-et-Vilaine", "Ille-et-Vilaine"), "ru": ("Иль и Вилен", "Иль и Вилен")},
    "36"   : {"fr": ("Indre", "Indre (département)"), "en": ("Indre", "Indre"), "ru": ("Эндр", "Эндр (департамент)")},
    "37"   : {"fr": ("Indre-et-Loire", "Indre-et-Loire"), "en": ("Indre-et-Loire", "Indre-et-Loire"), "ru": ("Эндр и Луа́ра", "Эндр и Луара")},
    "38"   : {"fr": ("Isère", "Isère (département)"), "en": ("Isère", "Isère"), "ru": ("Изе́р", "Изер (департамент)")},
    "39"   : {"fr": ("Jura", "Jura (département)"), "en": ("Jura", "Jura (department)"), "ru": ("Юра́", "Юра (департамент)")},
    "40"   : {"fr": ("Landes", "Landes (département)"), "en": ("Landes", "Landes (department)"), "ru": ("Ланды", "Ланды (департамент)")},
    "41"   : {"fr": ("Loir-et-Cher", "Loir-et-Cher"), "en": ("Loir-et-Cher", "Loir-et-Cher"), "ru": ("Луар и Шер", "Луар и Шер")},
    "42"   : {"fr": ("Loire", "Loire (département)"), "en": ("Loire", "Loire (department)"), "ru": ("Луа́ра", "Луара (департамент)")},
    "43"   : {"fr": ("Haute-Loire", "Haute-Loire"), "en": ("Haute-Loire", "Haute-Loire"), "ru": ("Верхняя Луа́ра", "Верхняя Луара")},
    "44"   : {"fr": ("Loire-Atlantique", "Loire-Atlantique"), "en": ("Loire-Atlantique", "Loire-Atlantique"), "ru": ("Атлантическая Луа́ра", "Атлантическая Луара")},
    "45"   : {"fr": ("Loiret", "Loiret (département)"), "en": ("Loiret", "Loiret"), "ru": ("Луаре́", "Луаре")},
    "46"   : {"fr": ("Lot", "Lot (département)"), "en": ("Lot", "Lot (department)"), "ru": ("Ло (Лот)", "Ло (департамент)")},
    "47"   : {"fr": ("Lot-et-Garonne", "Lot-et-Garonne"), "en": ("Lot-et-Garonne", "Lot-et-Garonne"), "ru": ("Ло и Гаро́нна", "Ло и Гаронна")},
    "48"   : {"fr": ("Lozère", "Lozère (département)"), "en": ("Lozère", "Lozère"), "ru": ("Лозе́р", "Лозер")},
    "49"   : {"fr": ("Maine-et-Loire", "Maine-et-Loire"), "en": ("Maine-et-Loire", "Maine-et-Loire"), "ru": ("Мен и Луа́ра", "Мен и Луара")},
    "50"   : {"fr": ("Manche", "Manche (département)"), "en": ("Manche", "Manche"), "ru": ("Манш", "Манш")},
    "51"   : {"fr": ("Marne", "Marne (département)"), "en": ("Marne", "Marne (department)"), "ru": ("Ма́рна", "Марна (департамент)")},
    "52"   : {"fr": ("Haute-Marne", "Haute-Marne"), "en": ("Haute-Marne", "Haute-Marne"), "ru": ("Верхняя Ма́рна", "Верхняя Марна")},
    "53"   : {"fr": ("Mayenne", "Mayenne (département)"), "en": ("Mayenne", "Mayenne"), "ru": ("Майе́н", "Майен (департамент)")},
    "54"   : {"fr": ("Meurthe-et-Moselle", "Meurthe-et-Moselle"), "en": ("Meurthe-et-Moselle", "Meurthe-et-Moselle"), "ru": ("Мёрт и Мозе́ль", "Мёрт и Мозель")},
    "55"   : {"fr": ("Meuse", "Meuse (département)"), "en": ("Meuse", "Meuse (department)"), "ru": ("Мёз", "Мёз (департамент)")},
    "56"   : {"fr": ("Morbihan", "Morbihan"), "en": ("Morbihan", "Morbihan"), "ru": ("Морбиан", "Морбиан")},
    "57"   : {"fr": ("Moselle", "Moselle (département)"), "en": ("Moselle", "Moselle (department)"), "ru": ("Мозе́ль", "Мозель (департамент)")},
    "58"   : {"fr": ("Nièvre", "Nièvre (département)"), "en": ("Nièvre", "Nièvre"), "ru": ("Ньевр", "Ньевр")},
    "59"   : {"fr": ("Nord", "Nord (département français)"), "en": ("Nord", "Nord (French department)"), "ru": ("Нор", "Нор (департамент)")},
    "60"   : {"fr": ("Oise", "Oise (département)"), "en": ("Oise", "Oise"), "ru": ("Уа́за", "Уаза (департамент)")},
    "61"   : {"fr": ("Orne", "Orne (département)"), "en": ("Orne", "Orne"), "ru": ("Орн", "Орн (департамент)")},
    "62"   : {"fr": ("Pas-de-Calais", "Pas-de-Calais"), "en": ("Pas-de-Calais", "Pas-de-Calais"), "ru": ("Па-де-Кале́", "Па-де-Кале (департамент)")},
    "63"   : {"fr": ("Puy-de-Dôme", "Puy-de-Dôme"), "en": ("Puy-de-Dôme", "Puy-de-Dôme"), "ru": ("Пюи́-де-Дом", "Пюи-де-Дом")},
    "64"   : {"fr": ("Pyrénées-Atlantiques", "Pyrénées-Atlantiques"), "en": ("Pyrénées-Atlantiques", "Pyrénées-Atlantiques"), "ru": ("Атлантические Пирене́и", "Атлантические Пиренеи")},
    "65"   : {"fr": ("Hautes-Pyrénées", "Hautes-Pyrénées"), "en": ("Hautes-Pyrénées", "Hautes-Pyrénées"), "ru": ("Верхние Пирене́и", "Верхние Пиренеи")},
    "66"   : {"fr": ("Pyrénées-Orientales", "Pyrénées-Orientales"), "en": ("Pyrénées-Orientales", "Pyrénées-Orientales"), "ru": ("Восточные Пирене́и", "Восточные Пиренеи")},
    "67"   : {"fr": ("Bas-Rhin", "Bas-Rhin"), "en": ("Bas-Rhin", "Bas-Rhin"), "ru": ("Нижний Рейн", "Нижний Рейн (департамент)")},
    "68"   : {"fr": ("Haut-Rhin", "Haut-Rhin"), "en": ("Haut-Rhin", "Haut-Rhin"), "ru": ("Верхний Рейн", "Верхний Рейн")},
    "69"   : {"fr": ("Rhône", "Rhône (département)"), "en": ("Rhône", "Rhône (department)"), "ru": ("Ро́на", "Рона (департамент)")},
    "70"   : {"fr": ("Haute-Saône", "Haute-Saône"), "en": ("Haute-Saône", "Haute-Saône"), "ru": ("Верхняя Со́на", "Верхняя Сона")},
    "71"   : {"fr": ("Saône-et-Loire", "Saône-et-Loire"), "en": ("Saône-et-Loire", "Saône-et-Loire"), "ru": ("Со́на и Луа́ра", "Сона и Луара")},
    "72"   : {"fr": ("Sarthe", "Sarthe (département)"), "en": ("Sarthe", "Sarthe"), "ru": ("Сарта", "Сарта (департамент)")},
    "73"   : {"fr": ("Savoie", "Savoie (département)"), "en": ("Savoie", "Savoie"), "ru": ("Саво́йя", "Савойя (департамент)")},
    "74"   : {"fr": ("Haute-Savoie", "Haute-Savoie"), "en": ("Haute-Savoie", "Haute-Savoie"), "ru": ("Верхняя Саво́йя", "Верхняя Савойя")},
    "75"   : {"fr": ("Paris", "Paris"), "en": ("Paris", "Paris"), "ru": ("Пари́ж", "Париж")},
    "76"   : {"fr": ("Seine-Maritime", "Seine-Maritime"), "en": ("Seine-Maritime", "Seine-Maritime"), "ru": ("Приморская Се́на", "Приморская Сена")},
    "77"   : {"fr": ("Seine-et-Marne", "Seine-et-Marne"), "en": ("Seine-et-Marne", "Seine-et-Marne"), "ru": ("Се́на и Ма́рна", "Сена и Марна")},
    "78"   : {"fr": ("Yvelines", "Yvelines"), "en": ("Yvelines", "Yvelines"), "ru": ("Ивелин", "Ивелин")},
    "79"   : {"fr": ("Deux-Sèvres", "Deux-Sèvres"), "en": ("Deux-Sèvres", "Deux-Sèvres"), "ru": ("Дё-Севр", "Дё-Севр")},
    "80"   : {"fr": ("Somme", "Somme (département)"), "en": ("Somme", "Somme (department)"), "ru": ("Со́мма", "Сомма (департамент)")},
    "81"   : {"fr": ("Tarn", "Tarn (département)"), "en": ("Tarn", "Tarn (department)"), "ru": ("Тарн", "Тарн (департамент)")},
    "82"   : {"fr": ("Tarn-et-Garonne", "Tarn-et-Garonne"), "en": ("Tarn-et-Garonne", "Tarn-et-Garonne"), "ru": ("Тарн и Гаро́нна", "Тарн и Гаронна")},
    "83"   : {"fr": ("Var", "Var (département)"), "en": ("Var", "Var (department)"), "ru": ("Вар", "Вар (департамент)")},
    "84"   : {"fr": ("Vaucluse", "Vaucluse (département)"), "en": ("Vaucluse", "Vaucluse"), "ru": ("Воклю́з", "Воклюз")},
    "85"   : {"fr": ("Vendée", "Vendée (département)"), "en": ("Vendée", "Vendée"), "ru": ("Ванде́я", "Вандея")},
    "86"   : {"fr": ("Vienne", "Vienne (département)"), "en": ("Vienne", "Vienne (department)"), "ru": ("Вье́нна", "Вьенна (департамент)")},
    "87"   : {"fr": ("Haute-Vienne", "Haute-Vienne"), "en": ("Haute-Vienne", "Haute-Vienne"), "ru": ("Верхняя Вье́нна", "Верхняя Вьенна")},
    "88"   : {"fr": ("Vosges", "Vosges (département)"), "en": ("Vosges", "Vosges (department)"), "ru": ("Воге́зы", "Вогезы (департамент)")},
    "89"   : {"fr": ("Yonne", "Yonne (département)"), "en": ("Yonne", "Yonne"), "ru": ("Йо́нна", "Йонна")},
    "90"   : {"fr": ("Territoire de Belfort", "Territoire de Belfort"), "en": ("Territoire de Belfort", "Territoire de Belfort"), "ru": ("Территория Бельфо́р", "Территория Бельфор")},
    "91"   : {"fr": ("Essonne", "Essonne (département)"), "en": ("Essonne", "Essonne"), "ru": ("Эсон", "Эсон (департамент)")},
    "92"   : {"fr": ("Hauts-de-Seine", "Hauts-de-Seine"), "en": ("Hauts-de-Seine", "Hauts-de-Seine"), "ru": ("О-де-Сен", "О-де-Сен")},
    "93"   : {"fr": ("Seine-Saint-Denis", "Seine-Saint-Denis"), "en": ("Seine-Saint-Denis", "Seine-Saint-Denis"), "ru": ("Сен-Сен-Дени́", "Сен-Сен-Дени")},
    "94"   : {"fr": ("Val-de-Marne", "Val-de-Marne"), "en": ("Val-de-Marne", "Val-de-Marne"), "ru": ("Валь-де-Марн", "Валь-де-Марн")},
    "95"   : {"fr": ("Val-d'Oise", "Val-d'Oise"), "en": ("Val-d'Oise", "Val-d'Oise"), "ru": ("Валь-д’Уаз", "Валь-д’Уаз")},
    "971"   : {"fr": ("Guadeloupe", "Guadeloupe"), "en": ("Guadeloupe", "Guadeloupe"), "ru": ("Гваделу́па", "Гваделупа")},
    "972"   : {"fr": ("Martinique", "Martinique"), "en": ("Martinique", "Martinique"), "ru": ("Мартини́ка", "Мартиника")},
    "973"   : {"fr": ("Guyane", "Guyane"), "en": ("French Guiana", "French Guiana"), "ru": ("Гвиа́на (Французская Гвиа́на)", "Гвиана (департамент Франции)")},
    "974"   : {"fr": ("La Réunion", "La Réunion"), "en": ("Réunion", "Réunion"), "ru": ("Реюньо́н", "Реюньон")},
    "976"   : {"fr": ("Mayotte", "Mayotte"), "en": ("Mayotte", "Mayotte"), "ru": ("Майо́тта", "Майотта")}
}

    # "69D"   : {"fr": ("Rhône", "Rhône (département)"), "en": ("Rhône", "Rhône (department)"), "ru": ("Ро́на", "Рона (департамент)")},
    # "69M"   : {"fr": ("Métropole de Lyon (Grand Lyon)", "Métropole de Lyon"), "en": ("Lyon Metropolis (Grand Lyon)", "Metropolis of Lyon"), "ru": ("Лио́нская метрополия (Большой Лио́н)", "Лионская метрополия")},

In [12]:
# create code for placing info in Wikipedia
def create_table(df, file_header, lang='ru'):

    def if_value(x, prec=1):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=1, *, add_par=''):  # change_value
        return f'style="padding-right:1.5ex;{add_par}"| —' if math.isnan(x) else \
               f'style="padding-right:1.5ex;color:darkgreen;{add_par}"| {x:0.{prec}f}' if x>0 else \
               f'style="padding-right:1.5ex;color:crimson;{add_par}"| −{-x:0.{prec}f}' if x<0 else \
               f'style="padding-right:1.5ex;color:darkgray;{add_par}"| {x:0.{prec}f}'
    
    def chval_bold(x, prec=1, *, add_par=''):  # change_value
        return ' —' if math.isnan(x) else \
               f'style="padding-right:1.5ex;color:darkgreen;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="padding-right:1.5ex;color:crimson;{add_par}"| \'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="padding-right:1.5ex;color:darkgray;{add_par}"| \'\'\'{x:0.{prec}f}\'\'\''
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name in ["M", "F"]:  # '66', '11', '34', '30', '13', '83', '06'
             st += '\n' + '|-class=static-row-header\n' + \
                  f'| \'\'\'{ser.name}\'\'\' ' + \
                  f'|| \'\'\'{dd_replacement[ser.name][lang][0]}\'\'\' ' + \
                  f'||style="background:#eaf3ff;"| \'\'\'{if_value(ser["2015_male"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["2015_female"])}\'\'\' ' + \
                  f'||style="background:#fff8dc;"| \'\'\'{if_value(ser["2015_fΔm"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2015_ar_mean"], prec=2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2015→2019"], prec=2, add_par="border-left-width:2px;")} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| \'\'\'{if_value(ser["2019_male"])}\'\'\' ' + \
                  f'||style=";background:#fee7f6;"| \'\'\'{if_value(ser["2019_female"])}\'\'\' ' + \
                  f'||style="background:#fff8dc;"| \'\'\'{if_value(ser["2019_fΔm"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2019_ar_mean"], prec=2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2019→2024"], prec=2, add_par="border-left-width:2px;")} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| \'\'\'{if_value(ser["2024_male"])}\'\'\' ' + \
                  f'||style="background:#fee7f6;"| \'\'\'{if_value(ser["2024_female"])}\'\'\' ' + \
                  f'||style="background:#fff8dc;"| \'\'\'{if_value(ser["2024_fΔm"])}\'\'\' ' + \
                  f'|| \'\'\'{if_value(ser["2024_ar_mean"], prec=2)}\'\'\' ' + \
                  f'||{chval_bold(ser["2015→2024"], prec=2, add_par="border-left-width:2px;")}'
        else:
            dep_link = dd_replacement[ser.name][lang][1]
            dep_visible = dd_replacement[ser.name][lang][0]
            dep_inserted = dep_link if dep_link == dep_visible else f"{dep_link}|{dep_visible}"
            st += '\n' + '|-\n' + \
                  f'| {ser.name} ' + \
                  f'|| [[{dep_inserted}]] ' + \
                  f'||style="background:#eaf3ff;"| {if_value(ser["2015_male"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["2015_female"])} ' + \
                  f'||style="background:#fff8dc;"| {if_value(ser["2015_fΔm"])} ' + \
                  f'|| {if_value(ser["2015_ar_mean"], prec=2)} ' + \
                  f'||{chval(ser["2015→2019"], prec=2, add_par="border-left-width:2px;")} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| {if_value(ser["2019_male"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["2019_female"])} ' + \
                  f'||style="background:#fff8dc;"| {if_value(ser["2019_fΔm"])} ' + \
                  f'|| {if_value(ser["2019_ar_mean"], prec=2)} ' + \
                  f'||{chval(ser["2019→2024"], prec=2, add_par="border-left-width:2px;")} ' + \
                  f'||style="background:#eaf3ff;border-left-width:2px;"| {if_value(ser["2024_male"])} ' + \
                  f'||style="background:#fee7f6;"| {if_value(ser["2024_female"])} ' + \
                  f'||style="background:#fff8dc;"| {if_value(ser["2024_fΔm"])} ' + \
                  f'|| {if_value(ser["2024_ar_mean"], prec=2)} ' + \
                  f'||{chval(ser["2015→2024"], prec=2, add_par="border-left-width:2px;")}'

    if lang == 'ru':
        st = re.sub('(?<=\\d)\\.(?=\\d)', ',', st)  # replace . to comma, if this . is between two digits
        st = st.replace('padding-right:1,5ex;', 'padding-right:1.5ex;')
        st = st.replace("[[Рона (департамент)|Ро́на]]", "[[Рона (департамент)|Ро́на]], [[Лионская метрополия|{{nobr|Лио́нская метрополия}}]]")
    elif lang == 'en':
        st = st.replace("[[Rhône (department)|Rhône]]", "[[Rhône (department)|Rhône]], [[Metropolis of Lyon|{{nobr|Lyon Metropolis}}]]")

    st = table_header + st + '\n|}'
    
    # gray color for missing values
    st = st.replace(';"| —', ';color:silver;"| —') \
           .replace('|| —', '||style="color:silver;"| —')

    return st


table_code = create_table(df, file_header='French_header_ru -departments -2024_v2.txt', lang='ru')

# write the code to file
with open('output/Table code for French departments -ru_v2.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

In [13]:
table_code = create_table(df, file_header='French_header_en -departments -2024_v2.txt', lang='en')

# write the code to file
with open('output/Table code for French departments -en_v2.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br />
<br />

range of values:<br />
male:<br />
76.4 - 82.8 → Δ6.4 (\~7)<br />
<br />
female:<br />
82.9 - 87.4 → Δ4.5 (\~6)